In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import pacf
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Test forecast with daily data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/daily/processed_daily.csv',index_col=0)
data['date'] = pd.to_datetime(data['date'])
data.set_index(data['date'], inplace=True)
data.sort_index(inplace=True)
data.rename_axis('index',inplace=True)
data.head()

,date,value
index,,
2019-07-14,2019-07-14,6.0
2019-07-15,2019-07-15,7.0
2019-07-16,2019-07-16,6.0
2019-07-17,2019-07-17,6.0
2019-07-18,2019-07-18,7.0


In [ ]:
data.tail()

,date,value
index,,
2023-01-28,2023-01-28,27.0
2023-01-29,2023-01-29,153181.0
2023-01-30,2023-01-30,541232.0
2023-01-31,2023-01-31,712411.0
2023-02-01,2023-02-01,1.0


In [ ]:
data.shape

(1299, 2)

In [ ]:
import numpy as np

def approximate_entropy(data, m, r):
    """
    Compute the approximate entropy (ApEn) of a time series data.

    Parameters:
        data (array-like): The time series data.
        m (int): The embedding dimension (usually set to 2).
        r (float): The tolerance parameter.

    Returns:
        float: The approximate entropy value.
    """
    N = len(data)
    phi = np.zeros(2)
    for i in range(2):
        phi[i] = _phi(data, m + i, r, N)
    return phi[0] - phi[1]

def _phi(data, m, r, N):
    """
    Helper function to compute phi values for approximate entropy.
    """
    C = np.zeros(N - m + 1)
    count = np.zeros(N - m + 1)
    for i in range(N - m + 1):
        for j in range(N - m + 1):
            if np.max(np.abs(data[i:i+m] - data[j:j+m])) <= r:
                count[i] += 1
                if abs(data[i+m-1] - data[j+m-1]) <= r:
                    C[i] += 1
        if count[i] != 0:
            C[i] /= count[i]
    return np.mean(np.log(C[np.nonzero(C)]))

def map_to_score(apen_value, min_apen, max_apen, min_score, max_score):
    """
    Map ApEn value to a forecastability score.
    """
    return (max_score - min_score) * (1 - (apen_value - min_apen) / (max_apen - min_apen)) + min_score

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from scipy.stats import zscore

daily_df = pd.read_csv("/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/daily/processed_daily.csv")
print(daily_df.head(10))
n_splits = 100
tscv = TimeSeriesSplit(n_splits=n_splits)

X = daily_df.drop(['value'], axis=1)
y = daily_df['value']

results_df = pd.DataFrame(columns =[
    'Mean',
    'Variance',
    'Skewness',
    'Kurtosis',
    'Trend Mean',
    'Seasonal Mean',
    'Residual Mean',
])
track_log = {}

for start_index, (train_index, test_index) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    sample_df = pd.concat([X_train, y_train], axis=1)
    sample_df['value'].index = pd.to_datetime(sample_df['value'].index).to_period('D')

    # Calculate statistical features
    mean = sample_df['value'].mean()
    variance = sample_df['value'].var()
    skewness = skew(sample_df['value'])
    kurt = kurtosis(sample_df['value'])

    # Calculate seasonal features
    result = seasonal_decompose(sample_df['value'], model='additive', period=24)
    trend_mean = result.trend.mean()
    seasonal_mean = result.seasonal.mean()
    residual_mean = result.resid.mean()

    m = 2  # Embedding dimension
    r = 0.2 * np.std(sample_df['value'])  # Tolerance parameter (fraction of standard deviation)
    apen_value = approximate_entropy(sample_df['value'], m, r)
    min_apen = 0.0  # Minimum observed ApEn value
    max_apen = 1.0  # Maximum observed ApEn value
    min_score = 0  # Minimum forecastability score
    max_score = 10  # Maximum forecastability score

    forecastability_score = map_to_score(apen_value, min_apen, max_apen, min_score, max_score)

    result_dict = {
          'Mean': mean,
          'Variance': variance,
          'Skewness': skewness,
          'Kurtosis': kurt,
          'Trend Mean': trend_mean,
          'Seasonal Mean': seasonal_mean,
          'Residual Mean': residual_mean,
          'Forecast_Score': forecastability_score
    }
    results_df = results_df.append(result_dict, ignore_index=True)

# Generate hypothetical RMSE and R^2 values based on forecastability score mean and standard deviation
forecast_score_mean = np.mean(data["Forecast_Score"])
forecast_score_std = np.std(data["Forecast_Score"])
thresholds = forecast_score_mean+ 1.96 *forecast_score_std
filtered_df = results_df[results_df['Forecast_Score'] >= thresholds]
print("Threshold daily:  ",thresholds)
print(filtered_df.head())
filtered_df.to_csv('/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/daily/results_daily.csv',index=False)

   Unnamed: 0        date  value
0           0  2019-07-14    6.0
1           1  2019-07-15    7.0
2           2  2019-07-16    6.0
3           3  2019-07-17    6.0
4           4  2019-07-18    7.0
5           5  2019-07-19    6.0
6           6  2019-07-20    7.0
7           7  2019-07-21    6.0
8           8  2019-07-22    6.0
9           9  2019-07-23    6.0


<ipython-input-17-c717ba456e43>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
<ipython-input-17-c717ba456e43>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
<ipython-input-17-c717ba456e43>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
<ipython-input-17-c717ba456e43>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
<ipython-input-17-c717ba456e43>:62: FutureWarning: The frame.app

KeyboardInterrupt: 

## Calculate forecastibility for rest of data

In [ ]:
folder_path = ["/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/hourly/processed_hourly.csv",
               "/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/monthly/processed_monthly.csv",
               "/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/weekly/processed_weekly.csv"]
temp = []

for index , folder in enumerate(folder_path):

  df = pd.read_csv(folder)
  print(df.info())
  df['date'] = pd.to_datetime(df['date'])
  df.set_index(df['date'], inplace=True)
  df.sort_index(inplace=True)
  df.rename_axis('index',inplace=True)


  n_splits = 50

  tscv = TimeSeriesSplit(n_splits=n_splits)

  X = df.drop(['value'], axis=1)
  y = df['value']

  results_df = pd.DataFrame(columns =[
    'Mean',
    'Variance',
    'Skewness',
    'Kurtosis',
    'Trend Mean',
    'Seasonal Mean',
    'Residual Mean',
  ])

  track_log = {}


  for start_index, (train_index, test_index) in enumerate(tscv.split(X)):
      X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      y_train, y_test = y[train_index], y[test_index]
      sample_df = pd.concat([X_train, y_train], axis=1)
      sample_df['value'].index = pd.to_datetime(sample_df['value'].index).to_period('D')

      # Calculate statistical features
      mean = sample_df['value'].mean()
      variance = sample_df['value'].var()
      skewness = skew(sample_df['value'])
      kurt = kurtosis(sample_df['value'])


      # Calculate time-based features
      try:
          result = seasonal_decompose(
              sample_df["value"], model="multiplicative", extrapolate_trend="freq"
          )
          trend_mean = result.trend.mean()
          seasonal_mean = result.seasonal.mean()
          residual_mean = result.resid.mean()
      except:
          result = seasonal_decompose(
              sample_df["value"], model="additive", extrapolate_trend="freq", period=2
          )
          trend_mean = result.trend.mean()
          seasonal_mean = result.seasonal.mean()
          residual_mean = result.resid.mean()

      cv = sample_df['value'].std()/sample_df['value'].mean()
      forecastability_score = (1 - (cv / (1 + cv)))
      min_fs = 0
      max_fs = 10
      normalized_fs = (forecastability_score-min_fs)/(max_fs-min_fs)
      scaled_forecast_score = normalized_fs * (max_fs - min_fs) + min_fs
      forecast_score = max(min(scaled_forecast_score, 10), 0)*10

      result_dict = {
            'Mean': mean,
            'Variance': variance,
            'Skewness': skewness,
            'Kurtosis': kurt,
            'Trend Mean': trend_mean,
            'Seasonal Mean': seasonal_mean,
            'Residual Mean': residual_mean,
            'Forecast_Score': forecast_score
      }


      results_df = results_df.append(result_dict, ignore_index=True)

  forecast_score_mean = np.mean(results_df['Forecast_Score'])
  forecast_score_std = np.std(results_df['Forecast_Score'])
  z_scores = zscore(results_df['Forecast_Score'])
  lb = forecast_score_mean - 1.5*forecast_score_std
  ub = forecast_score_mean + 1.5 * forecast_score_std
  filtered_df = results_df[(results_df['Forecast_Score'] >= lb) & (results_df['Forecast_Score'] <= ub)]
  print(filtered_df.head(10))
  break
  temp.append(track_log)
  if "hourly" in folder:
    print("Threshold hourly : ", lb,ub)
    filtered_df.to_csv('/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/hourly/results_hourly.csv',index=False)

  if "weekly" in folder:
    print("Threshold weekly : ", lb,ub)
    filtered_df.to_csv('/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/weekly/results_weekly.csv',index=False)

  if "monthly" in folder:
    print("Threshold monthly : ", lb,ub)
    filtered_df.to_csv('/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/monthly/results_monthly.csv',index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1299 entries, 0 to 1298
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    1299 non-null   object 
 1   value   1299 non-null   float64
dtypes: float64(1), object(1)
memory usage: 20.4+ KB
None


<ipython-input-7-42b40c35fa2d>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
<ipython-input-7-42b40c35fa2d>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
<ipython-input-7-42b40c35fa2d>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
<ipython-input-7-42b40c35fa2d>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
<ipython-input-7-42b40c35fa2d>:85: FutureWarning: The frame.append m

        Mean    Variance  Skewness  Kurtosis  Trend Mean  Seasonal Mean  \
0  24.063210  647.942598 -0.445462 -1.764407   24.949745       0.054443   
1  29.447480  640.094571 -0.713868 -1.283695   29.570319       0.000000   
2  32.889586  748.767156 -0.450547 -1.197792   32.501983       0.011628   
3  36.596564  799.071060 -0.504298 -1.181259   36.681910       0.000000   
4  37.840598  792.833271 -0.550615 -1.190938   37.781853       0.013614   
5  37.702572  800.403428 -0.503713 -1.284352   38.111431       0.000000   
6  38.811564  817.541717 -0.487403 -1.180511   38.882619       0.001446   
7  38.879662  814.368216 -0.494954 -1.230523   38.682258       0.000000   
8  38.382918  829.406250 -0.449954 -1.319854   38.237645       0.001299   
9  38.515048  869.850960 -0.403114 -1.390038   38.658312       0.000000   

   Residual Mean  Forecast_Score  
0      -0.940978        4.859495  
1      -0.122839        5.378774  
2       0.375975        5.458563  
3      -0.085346        5.642010  

<ipython-input-7-42b40c35fa2d>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
<ipython-input-7-42b40c35fa2d>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
<ipython-input-7-42b40c35fa2d>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
<ipython-input-7-42b40c35fa2d>:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(result_dict, ignore_index=True)
<ipython-input-7-42b40c35fa2d>:85: FutureWarning: The frame.append m

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DataGenieHackathon/Sample Time Series/hourly/results_hourly.csv')
df.head(10)

,Unnamed: 0,Mean,Variance,Skewness,Kurtosis,Trend Mean,Seasonal Mean,Residual Mean,Forecast_Score
0,0,24.063210,647.942598,-0.445462,-1.764407,24.949745,0.054443,-0.940978,4.859495
1,1,29.447480,640.094571,-0.713868,-1.283695,29.570319,0.000000,-0.122839,5.378774
2,2,32.889586,748.767156,-0.450547,-1.197792,32.501983,0.011628,0.375975,5.458563
3,3,36.596564,799.071060,-0.504298,-1.181259,36.681910,0.000000,-0.085346,5.642010
4,4,37.840598,792.833271,-0.550615,-1.190938,37.781853,0.013614,0.045131,5.733607
5,5,37.702572,800.403428,-0.503713,-1.284352,38.111431,0.000000,-0.408859,5.713033
6,6,38.811564,817.541717,-0.487403,-1.180511,38.882619,0.001446,-0.072501,5.758029
7,7,38.879662,814.368216,-0.494954,-1.230523,38.682258,0.000000,0.197404,5.767058
8,8,38.382918,829.406250,-0.449954,-1.319854,38.237645,0.001299,0.143973,5.713246
9,9,38.515048,869.850960,-0.403114,-1.390038,38.658312,0.000000,-0.143264,5.663288
